<a href="https://colab.research.google.com/github/branley1/smol-course/blob/main/1_instruction_tuning/notebooks/sft_finetuning_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p>
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [1]:
# Install the requirements in Google Colab
!pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face
from huggingface_hub import login
!huggingface-cli login

login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineG

In [2]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# Generate with the base model

Here we will try out the base model which does not have a chat template.

In [3]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a


## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [14]:
# Load a sample dataset
from datasets import load_dataset, Dataset

# Tokenizer setup
tokenizer = AutoTokenizer.from_pretrained("gpt2")
if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token

# Define datasets
# Smoltalk dataset
ds_smoltalk = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

# Process Smoltalk dataset for TRL chat templates
def process_smoltalk(sample):
    # Convert Smoltalk conversation to chat format
    messages = [{"role": msg["role"], "content": msg["content"]} for msg in sample["messages"]]
    return {"messages": messages}

# Apply the processing
ds_smoltalk = ds_smoltalk.map(process_smoltalk, remove_columns=ds_smoltalk["train"].column_names)
display(ds_smoltalk, "ds_smoltalk")

# BigCode dataset
ds_big_code = load_dataset(path="bigcode/the-stack-smol-xl", data_dir="data/python")

# Process BigCode dataset for TRL chat templates
def process_big_code(sample):
    # For BigCode, structure the code snippet into a chat format
    messages = [
        {"role": "system", "content": "You are a programming assistant specializing in Python."},
        {"role": "user", "content": "Write or explain the following Python code:"},
        {"role": "assistant", "content": sample["content"]},
    ]
    return {"messages": messages}

# Convert the generator to a list before creating the Dataset
processed_big_code = [process_big_code(sample) for sample in ds_big_code["train"]]
ds_big_code_train = Dataset.from_list(processed_big_code)  # Create a new Dataset object
ds_big_code["train"] = ds_big_code_train  # Replace the original 'train' split with the processed one
display(ds_big_code, "ds_big_code")

# CodeParrot dataset
languages_to_keep = ["Python", "C++", "C", "HTML", "CSS", "SQL", "TypeScript", "TeX"]
ds_code_parrot = load_dataset("codeparrot/github-code", streaming=True, split="train").filter(
    lambda x: x["language"] in languages_to_keep)

# Convert processed CodeParrot datatset into a generator
processed_code_parrot = (process_code_parrot(sample) for sample in ds_code_parrot)

# Process CodeParrot dataset for TRL chat templates
def process_code_parrot(sample):
    messages = [
        {"role": "system", "content": f"You are a programming assistant specializing in {sample['language']}."},
        {"role": "user", "content": "Write or explain the following code snippet:"},
        {"role": "assistant", "content": sample["code"]},
    ]
    return {"messages": messages}

# Map the processing
processed_code_parrot = ds_code_parrot.map(process_code_parrot)
display(ds_code_parrot, "ds_code_parrot")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

'ds_smoltalk'

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 10000
    })
})

'ds_big_code'

IterableDataset({
    features: ['code', 'repo_name', 'path', 'language', 'license', 'size'],
    num_shards: 1126
})

'ds_code_parrot'

In [ ]:
# Tokenize datasets for training
def tokenize_dataset(dataset):
    def tokenize_function(sample):
        chat_str = ""
        for msg in sample["messages"]:
            chat_str += f"{msg['role'].capitalize()}: {msg['content']}\n"
        return tokenizer(chat_str, truncation=True, padding="max_length", max_length=512)
    return dataset.map(tokenize_function)

tokenized_smoltalk = tokenize_dataset(ds_smoltalk)
tokenized_big_code = tokenize_dataset(ds_big_code["train"])  # Assuming "train" split only
# Convert the processed_code_parrot map object to a list before passing it to Dataset.from_list
tokenized_code_parrot = tokenize_dataset(Dataset.from_list(list(processed_code_parrot)))

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [9]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_smoltalk,
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`

NameError: name 'ds' is not defined

## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [ ]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

In [ ]:
trainer.push_to_hub(tags=finetune_tags)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [ ]:
# Test the fine-tuned model on the same prompt

# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: use the fine-tuned to model generate a response, just like with the base example.

## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.